# Terrapay Transaction Monitoring Analysis

This notebook conducts a thorough analysis of Terrapay's transaction monitoring system to optimize rule efficiency and reduce false positives.

## Objectives

1. Identify KYC IDs that have alerted in the last 3 months across rules
2. Determine what percentage of KYC IDs alerted on multiple rules
3. Analyze true positive vs false positive rates for rules
4. Examine KYC breakage impact on rule efficiency
5. Identify redundant or overlapping rules
6. Generate recommendations for rule optimization

## 1. Setup and Data Loading

Let's import the necessary libraries and load the data.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from collections import defaultdict, Counter
import itertools
import os
from fuzzywuzzy import fuzz  # For name similarity matching
import networkx as nx
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram, linkage
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)

# Set visualization style
plt.style.use('ggplot')
sns.set(style="whitegrid")

# Create output directory for visualizations
os.makedirs('visualizations', exist_ok=True)

In [ ]:
# Load all data from Excel file
def load_data():
    """Load all data from Excel file"""
    # Read transaction data
    transaction_data = pd.read_excel('transaction_dummy_data_10k_final.xlsx', 
                                     sheet_name='transaction_dummy_data_10k')
    
    # Read metadata
    metadata = pd.read_excel('transaction_dummy_data_10k_final.xlsx', 
                             sheet_name='Sheet2')
    
    # Read rule descriptions
    rule_descriptions = pd.read_excel('transaction_dummy_data_10k_final.xlsx', 
                                     sheet_name='rule_description')
    
    # Convert dates to datetime if not already
    date_columns = ['transaction_date_time_local', 'created_at', 'closed_at', 
                    'kyc_sender_create_date', 'kyc_receiver_create_date',
                    'dob_sender', 'dob_receiver', 'self_closure_date']
    
    for col in date_columns:
        if col in transaction_data.columns:
            transaction_data[col] = pd.to_datetime(transaction_data[col])
    
    return transaction_data, metadata, rule_descriptions

# Load all data
transaction_data, metadata, rule_descriptions = load_data()

## 2. Initial Data Exploration

Let's explore the dataset to understand its structure and contents.

In [ ]:
# Basic exploratory analysis
print(f"Dataset shape: {transaction_data.shape}")
print(f"Data timeframe: {transaction_data['transaction_date_time_local'].min().date()} to {transaction_data['transaction_date_time_local'].max().date()}")

# Basic stats about the data
print("\nStatus distribution:")
status_counts = transaction_data['status'].value_counts()
print(status_counts)

# Plot status distribution
plt.figure(figsize=(10, 6))
sns.barplot(x=status_counts.index, y=status_counts.values)
plt.title('Alert Status Distribution')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

print("\nRule frequency distribution:")
print(transaction_data['rule_frequency'].value_counts())

print("\nRule pattern distribution:")
print(transaction_data['rule_pattern'].value_counts())

print("\nTop 10 most frequent alerting rules:")
top_rules = transaction_data['alert_rules'].value_counts().head(10)
print(top_rules)

# Plot top rules
plt.figure(figsize=(12, 6))
sns.barplot(x=top_rules.index, y=top_rules.values)
plt.title('Top 10 Most Frequent Alerting Rules')
plt.xticks(rotation=45)
plt.ylabel('Count')
plt.tight_layout()
plt.show()

# Distribution of triggered_on (sender vs receiver)
print("\nDistribution of triggered_on:")
print(transaction_data['triggered_on'].value_counts())

# Count of unique KYC IDs
print("\nUnique KYC IDs:")
print(f"Sender KYC IDs: {transaction_data['sender_kyc_id_no'].nunique()}")
print(f"Receiver KYC IDs: {transaction_data['receiver_kyc_id_no'].nunique()}")

# Display sample of rule descriptions
print("\nRule descriptions sample:")
print(rule_descriptions.head())

## 3. KYC Alert Overlap Analysis

In this section, we'll identify KYC IDs that have alerted across multiple rules to understand the overlap.

In [ ]:
def analyze_kyc_alert_overlap(transaction_data):
    """Analyze KYC IDs that have alerted across rules and determine overlap."""
    print("Analyzing KYC IDs that have alerted...")
    
    # Group alerts by KYC ID (based on triggered_on field)
    kyc_alerts = defaultdict(set)
    
    for idx, row in transaction_data.iterrows():
        if row['triggered_on'] == 'sender':
            kyc_id = row['sender_kyc_id_no']
        else:  # receiver
            kyc_id = row['receiver_kyc_id_no']
            
        kyc_alerts[kyc_id].add(row['alert_rules'])
    
    # Calculate statistics
    total_kyc_with_alerts = len(kyc_alerts)
    kyc_with_multiple_rules = sum(1 for rules in kyc_alerts.values() if len(rules) > 1)
    
    # Distribution of number of rules per KYC
    rule_count_per_kyc = [len(rules) for rules in kyc_alerts.values()]
    rule_count_distribution = pd.Series(rule_count_per_kyc).value_counts().sort_index()
    
    # Calculate overlap percentage
    overlap_percentage = (kyc_with_multiple_rules / total_kyc_with_alerts) * 100 if total_kyc_with_alerts > 0 else 0
    
    print(f"Total KYC IDs with alerts: {total_kyc_with_alerts}")
    print(f"KYC IDs alerting on multiple rules: {kyc_with_multiple_rules}")
    print(f"Percentage of KYC IDs alerting on multiple rules: {overlap_percentage:.2f}%")
    
    print("\nDistribution of number of rules per KYC ID:")
    print(rule_count_distribution)
    
    # Plot the distribution
    plt.figure(figsize=(10, 6))
    rule_count_distribution.plot(kind='bar')
    plt.title('Number of Rules Triggered per KYC ID')
    plt.xlabel('Number of Rules')
    plt.ylabel('Count of KYC IDs')
    plt.xticks(rotation=0)
    plt.tight_layout()
    plt.show()
    
    # Find co-occurring rules
    rule_pairs = []
    for rules in kyc_alerts.values():
        if len(rules) > 1:
            # Convert set to array for easier processing
            rule_list = list(rules)
            for i in range(len(rule_list)):
                for j in range(i+1, len(rule_list)):
                    rule_pairs.append((rule_list[i], rule_list[j]))
    
    # Count occurrences of each rule pair
    rule_pair_counts = pd.Series(rule_pairs).value_counts().head(15)
    
    print("\nTop 15 co-occurring rule pairs:")
    print(rule_pair_counts)
    
    # Create and plot co-occurrence matrix if we have rule pairs
    if rule_pairs:
        unique_rules = sorted(set(rule for pair in rule_pairs for rule in pair))
        
        # Only create a heatmap if not too large
        if len(unique_rules) <= 30:  
            cooccurrence_matrix = pd.DataFrame(0, index=unique_rules, columns=unique_rules)
            
            for r1, r2 in rule_pairs:
                cooccurrence_matrix.loc[r1, r2] += 1
                cooccurrence_matrix.loc[r2, r1] += 1
            
            plt.figure(figsize=(12, 10))
            sns.heatmap(cooccurrence_matrix, cmap="YlGnBu", annot=False)
            plt.title('Rule Co-occurrence Matrix')
            plt.tight_layout()
            plt.show()
    
    return kyc_alerts, rule_count_distribution, rule_pair_counts

In [ ]:
# Execute the KYC alert overlap analysis
kyc_alerts, rule_count_dist, rule_pairs = analyze_kyc_alert_overlap(transaction_data)

## 4. Rule Efficiency Analysis (True Positives vs False Positives)

Next, we'll analyze the efficiency of each rule based on true positive and false positive rates.

In [ ]:
def analyze_rule_efficiency(transaction_data, rule_descriptions):
    """Analyze the efficiency of rules based on true positive and false positive rates."""
    print("Analyzing rule efficiency based on true/false positives...")
    
    # Filter for closed alerts only (where investigation is complete)
    closed_alerts = transaction_data[transaction_data['status'].isin(['Closed TP', 'Closed FP'])]
    
    # Overall TP/FP rates
    true_positive_rate = len(closed_alerts[closed_alerts['status'] == 'Closed TP']) / len(closed_alerts) * 100
    false_positive_rate = len(closed_alerts[closed_alerts['status'] == 'Closed FP']) / len(closed_alerts) * 100
    
    print(f"Overall True Positive Rate: {true_positive_rate:.2f}%")
    print(f"Overall False Positive Rate: {false_positive_rate:.2f}%")
    
    # Create a performance dataframe for each rule
    rule_performance = closed_alerts.groupby('alert_rules').apply(
        lambda x: pd.Series({
            'Total': len(x),
            'TP': sum(x['status'] == 'Closed TP'),
            'FP': sum(x['status'] == 'Closed FP'),
            'TP_Rate': sum(x['status'] == 'Closed TP') / len(x) * 100 if len(x) > 0 else 0,
            'Frequency': x['rule_frequency'].iloc[0] if not x['rule_frequency'].empty else 'Unknown',
            'Pattern': x['rule_pattern'].iloc[0] if not x['rule_pattern'].empty else 'Unknown'
        })
    ).reset_index()
    
    # Merge with rule descriptions
    rule_performance = rule_performance.merge(
        rule_descriptions[['Rule no.', 'Rule description', 'Current threshold']], 
        left_on='alert_rules', 
        right_on='Rule no.', 
        how='left'
    ).drop('Rule no.', axis=1)
    
    # Sort by TP rate descending
    rule_performance_by_tp = rule_performance.sort_values('TP_Rate', ascending=False)
    
    print("\nRule performance by true positive rate (Top 10):")
    print(rule_performance_by_tp[['alert_rules', 'Total', 'TP', 'FP', 'TP_Rate', 'Frequency', 'Pattern']].head(10))
    
    print("\nRule performance by true positive rate (Bottom 10):")
    print(rule_performance_by_tp[['alert_rules', 'Total', 'TP', 'FP', 'TP_Rate', 'Frequency', 'Pattern']].tail(10))
    
    # Find inefficient rules (high volume, low TP rate)
    inefficient_rules = rule_performance[(rule_performance['Total'] > 50) & 
                                         (rule_performance['TP_Rate'] < 30)].sort_values('Total', ascending=False)
    
    print("\nInefficient rules (high volume, low TP rate):")
    print(inefficient_rules[['alert_rules', 'Total', 'TP', 'FP', 'TP_Rate', 'Frequency', 'Pattern']].head(10))
    
    # Analyze performance by pattern
    pattern_performance = rule_performance.groupby('Pattern').agg({
        'Total': 'sum',
        'TP': 'sum',
        'FP': 'sum'
    }).reset_index()
    
    pattern_performance['TP_Rate'] = pattern_performance['TP'] / pattern_performance['Total'] * 100
    pattern_performance = pattern_performance.sort_values('TP_Rate', ascending=False)
    
    print("\nPerformance by rule pattern:")
    print(pattern_performance)
    
    # Analyze performance by frequency
    frequency_performance = rule_performance.groupby('Frequency').agg({
        'Total': 'sum',
        'TP': 'sum',
        'FP': 'sum'
    }).reset_index()
    
    frequency_performance['TP_Rate'] = frequency_performance['TP'] / frequency_performance['Total'] * 100
    frequency_performance = frequency_performance.sort_values('TP_Rate', ascending=False)
    
    print("\nPerformance by rule frequency:")
    print(frequency_performance)
    
    # Plot TP rate by rule (top 20 by volume)
    top_rules_by_volume = rule_performance.sort_values('Total', ascending=False).head(20)
    plt.figure(figsize=(14, 7))
    bars = plt.bar(top_rules_by_volume['alert_rules'], top_rules_by_volume['TP_Rate'], 
            color=plt.cm.RdYlGn(top_rules_by_volume['TP_Rate']/100), alpha=0.7)
    plt.axhline(y=50, color='r', linestyle='--', label='50% TP Rate')
    plt.title('True Positive Rate for Top 20 Rules by Volume')
    plt.xlabel('Rule')
    plt.ylabel('True Positive Rate (%)')
    plt.xticks(rotation=45, ha='right')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
    # Plot by pattern
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Pattern', y='TP_Rate', data=pattern_performance, palette='viridis')
    plt.title('True Positive Rate by Rule Pattern')
    plt.xlabel('Pattern')
    plt.ylabel('True Positive Rate (%)')
    plt.tight_layout()
    plt.show()
    
    # Plot by frequency
    plt.figure(figsize=(10, 6))
    sns.barplot(x='Frequency', y='TP_Rate', data=frequency_performance, palette='plasma')
    plt.title('True Positive Rate by Rule Frequency')
    plt.xlabel('Frequency')
    plt.ylabel('True Positive Rate (%)')
    plt.tight_layout()
    plt.show()
    
    # Get list of true positives
    true_positives = transaction_data[transaction_data['status'] == 'Closed TP']
    
    # Extract unique KYC IDs with true positive alerts
    tp_kyc_ids = []
    for idx, row in true_positives.iterrows():
        if row['triggered_on'] == 'sender':
            tp_kyc_ids.append(row['sender_kyc_id_no'])
        else:  # receiver
            tp_kyc_ids.append(row['receiver_kyc_id_no'])
    
    unique_tp_kyc_ids = set(tp_kyc_ids)
    
    print(f"\nTotal true positive alerts: {len(true_positives)}")
    print(f"Number of unique KYC IDs with true positive alerts: {len(unique_tp_kyc_ids)}")
    
    return rule_performance, pattern_performance, frequency_performance, true_positives, unique_tp_kyc_ids

In [ ]:
# Execute the rule efficiency analysis
rule_performance, pattern_performance, frequency_performance, true_positives, tp_kyc_ids = analyze_rule_efficiency(transaction_data, rule_descriptions)

## 5. KYC Breakage Analysis

In this section, we'll analyze the KYC breakage issue where multiple KYC IDs exist for the same person.

In [ ]:
def analyze_kyc_breakage(transaction_data):
    """Analyze the KYC breakage issue (multiple KYC IDs for the same person)."""
    print("\nAnalyzing KYC breakage issue...")
    
    # Analyze sender KYC breakage
    print("\n=== Sender KYC Breakage Analysis ===")
    
    # Group by sender name and count KYC IDs
    sender_name_groups = transaction_data.groupby('sender_name_kyc_wise')['sender_kyc_id_no'].nunique().reset_index()
    sender_name_groups.columns = ['sender_name', 'kyc_id_count']
    
    # Distribution of KYC IDs per sender name
    sender_kyc_distribution = sender_name_groups['kyc_id_count'].value_counts().sort_index()
    
    print("Distribution of KYC IDs per unique sender name:")
    print(sender_kyc_distribution.head(10))
    
    # Calculate stats
    avg_kyc_per_sender = sender_name_groups['kyc_id_count'].mean()
    max_kyc_per_sender = sender_name_groups['kyc_id_count'].max()
    senders_with_multiple_kyc = sum(sender_name_groups['kyc_id_count'] > 1)
    percentage_senders_with_multiple_kyc = senders_with_multiple_kyc / len(sender_name_groups) * 100
    
    print(f"\nAverage KYC IDs per unique sender name: {avg_kyc_per_sender:.2f}")
    print(f"Maximum KYC IDs for a single sender name: {max_kyc_per_sender}")
    print(f"Senders with multiple KYC IDs: {senders_with_multiple_kyc} ({percentage_senders_with_multiple_kyc:.2f}%)")
    
    # Find senders with the most KYC IDs
    top_multiple_kyc_senders = sender_name_groups.sort_values('kyc_id_count', ascending=False).head(10)
    print("\nTop 10 senders with most KYC IDs:")
    print(top_multiple_kyc_senders)
    
    # Analyze receiver KYC breakage
    print("\n=== Receiver KYC Breakage Analysis ===")
    
    # Group by receiver name and count KYC IDs
    receiver_name_groups = transaction_data.groupby('receiver_name_kyc_wise')['receiver_kyc_id_no'].nunique().reset_index()
    receiver_name_groups.columns = ['receiver_name', 'kyc_id_count']
    
    # Distribution of KYC IDs per receiver name
    receiver_kyc_distribution = receiver_name_groups['kyc_id_count'].value_counts().sort_index()
    
    print("Distribution of KYC IDs per unique receiver name:")
    print(receiver_kyc_distribution.head(10))
    
    # Calculate stats
    avg_kyc_per_receiver = receiver_name_groups['kyc_id_count'].mean()
    max_kyc_per_receiver = receiver_name_groups['kyc_id_count'].max()
    receivers_with_multiple_kyc = sum(receiver_name_groups['kyc_id_count'] > 1)
    percentage_receivers_with_multiple_kyc = receivers_with_multiple_kyc / len(receiver_name_groups) * 100
    
    print(f"\nAverage KYC IDs per unique receiver name: {avg_kyc_per_receiver:.2f}")
    print(f"Maximum KYC IDs for a single receiver name: {max_kyc_per_receiver}")
    print(f"Receivers with multiple KYC IDs: {receivers_with_multiple_kyc} ({percentage_receivers_with_multiple_kyc:.2f}%)")
    
    # Plot KYC breakage distributions
    plt.figure(figsize=(12, 6))
    
    plt.subplot(1, 2, 1)
    sender_kyc_distribution.head(5).plot(kind='bar')
    plt.title('KYC IDs per Sender Name')
    plt.xlabel('Number of KYC IDs')
    plt.ylabel('Count of Sender Names')
    plt.xticks(rotation=0)
    
    plt.subplot(1, 2, 2)
    receiver_kyc_distribution.head(5).plot(kind='bar')
    plt.title('KYC IDs per Receiver Name')
    plt.xlabel('Number of KYC IDs')
    plt.ylabel('Count of Receiver Names')
    plt.xticks(rotation=0)
    
    plt.tight_layout()
    plt.show()
    
    # Analyze impact on receiver rules
    print("\n=== Impact on Receiver Rules ===")
    receiver_alerts = transaction_data[transaction_data['triggered_on'] == 'receiver']
    
    # Find receivers with multiple KYC IDs
    receivers_with_multiple_kyc_ids = receiver_name_groups[receiver_name_groups['kyc_id_count'] > 1]['receiver_name'].tolist()
    
    # Filter alerts for these receivers
    if receivers_with_multiple_kyc_ids:
        multiple_kyc_receiver_alerts = receiver_alerts[receiver_alerts['receiver_name_kyc_wise'].isin(receivers_with_multiple_kyc_ids)]
        
        # Count alerts by rule for these receivers
        if not multiple_kyc_receiver_alerts.empty:
            multi_kyc_rule_counts = multiple_kyc_receiver_alerts.groupby('alert_rules').size().sort_values(ascending=False)
            
            print("\nDistribution of rules for receivers with multiple KYC IDs:")
            print(multi_kyc_rule_counts.head(10))
            
            # Calculate percentage of alerts that might be affected by KYC breakage
            pct_receiver_alerts_affected = len(multiple_kyc_receiver_alerts) / len(receiver_alerts) * 100
            print(f"\nPercentage of receiver alerts potentially affected by KYC breakage: {pct_receiver_alerts_affected:.2f}%")
    
    return sender_name_groups, receiver_name_groups

In [ ]:
# Execute the KYC breakage analysis
sender_name_groups, receiver_name_groups = analyze_kyc_breakage(transaction_data)

## 6. Rule Overlap and Recommendations

Finally, we'll identify rule overlap and generate recommendations for optimization.

In [ ]:
def identify_rule_overlap_and_generate_recommendations(transaction_data, rule_descriptions, rule_performance, kyc_alerts):
    """Identify rule overlap and generate recommendations."""
    print("\nIdentifying rule overlap and generating recommendations...")
    
    # Create a correlation matrix for rules
    all_rules = transaction_data['alert_rules'].unique()
    cooccurrence_matrix = pd.DataFrame(0, index=all_rules, columns=all_rules)
    
    # Fill the co-occurrence matrix using the kyc_alerts dictionary
    for kyc_id, rules in kyc_alerts.items():
        rule_list = list(rules)
        for i in range(len(rule_list)):
            for j in range(i, len(rule_list)):
                cooccurrence_matrix.loc[rule_list[i], rule_list[j]] += 1
                if i != j:
                    cooccurrence_matrix.loc[rule_list[j], rule_list[i]] += 1
    
    # Convert to correlation matrix (Jaccard similarity)
    correlation_matrix = pd.DataFrame(0.0, index=all_rules, columns=all_rules)
    
    for i in all_rules:
        for j in all_rules:
            if i == j:
                correlation_matrix.loc[i, j] = 1.0
            else:
                intersection = cooccurrence_matrix.loc[i, j]
                union = cooccurrence_matrix.loc[i, i] + cooccurrence_matrix.loc[j, j] - intersection
                correlation_matrix.loc[i, j] = intersection / union if union > 0 else 0
    
    # Identify highly correlated rule pairs
    high_correlation_threshold = 0.7
    high_corr_pairs = []
    
    for i in range(len(all_rules)):
        for j in range(i+1, len(all_rules)):
            rule1, rule2 = all_rules[i], all_rules[j]
            corr = correlation_matrix.loc[rule1, rule2]
            if corr >= high_correlation_threshold:
                high_corr_pairs.append((rule1, rule2, corr))
    
    # Sort by correlation
    high_corr_pairs.sort(key=lambda x: x[2], reverse=True)
    
    print(f"\nIdentified {len(high_corr_pairs)} rule pairs with correlation >= {high_correlation_threshold}")
    if high_corr_pairs:
        print("Top 5 correlated pairs:")
        for rule1, rule2, corr in high_corr_pairs[:5]:
            print(f"  {rule1} and {rule2}: {corr:.3f}")
            
    # Visualize rule correlation as a heatmap
    if len(all_rules) <= 30:  # Only plot if not too many rules
        plt.figure(figsize=(12, 10))
        sns.heatmap(correlation_matrix, cmap='viridis', annot=False)
        plt.title('Rule Correlation Matrix')
        plt.tight_layout()
        plt.show()
    
    # Get rule information
    rule_info = rule_descriptions.set_index('Rule no.').to_dict()
    
    # Generate recommendations
    recommendations = []
    
    # 1. Consolidate highly correlated rules
    for rule1, rule2, corr in high_corr_pairs[:10]:
        if corr > 0.8:  # Very high correlation
            # Get rule details from rule_performance DataFrame
            rule1_perf = rule_performance[rule_performance['alert_rules'] == rule1]
            rule2_perf = rule_performance[rule_performance['alert_rules'] == rule2]
            
            if not rule1_perf.empty and not rule2_perf.empty:
                rule1_tp = rule1_perf.iloc[0]['TP_Rate']
                rule2_tp = rule2_perf.iloc[0]['TP_Rate']
                
                keep_rule = rule1 if rule1_tp >= rule2_tp else rule2
                remove_rule = rule2 if keep_rule == rule1 else rule1
                
                recommendations.append({
                    'Category': 'Consolidate Similar Rules',
                    'Rules': f"{rule1}, {rule2}",
                    'Action': f"Combine rules, keep {keep_rule}",
                    'Rationale': f"High correlation ({corr:.2f}), {keep_rule} has higher TP rate ({max(rule1_tp, rule2_tp):.1f}%)",
                    'Impact': 'Reduce alert volume while maintaining effectiveness'
                })
    
    # 2. Remove or modify inefficient rules
    inefficient_rules = rule_performance[(rule_performance['Total'] > 50) & 
                                          (rule_performance['TP_Rate'] < 30)].sort_values('Total', ascending=False)
    
    for _, rule in inefficient_rules.head(5).iterrows():
        recommendations.append({
            'Category': 'Optimize Inefficient Rules',
            'Rules': rule['alert_rules'],
            'Action': 'Increase threshold or consider removing',
            'Rationale': f"Low TP rate ({rule['TP_Rate']:.1f}%) with high volume ({rule['Total']} alerts)",
            'Impact': 'Significant reduction in false positives'
        })
    
    # 3. KYC breakage mitigation
    recommendations.append({
        'Category': 'KYC Breakage Mitigation',
        'Rules': 'All receiver-focused rules',
        'Action': 'Implement name/phone fuzzy matching before rule execution',
        'Rationale': 'Multiple KYC IDs for the same entity causing duplicate alerts',
        'Impact': 'More accurate entity identification, reduced false positives'
    })
    
    # 4. Pattern-based recommendations
    pattern_perf = rule_performance.groupby('Pattern')['TP_Rate'].mean().sort_values()
    
    for pattern, tp_rate in pattern_perf.items():
        if tp_rate < 40:
            pattern_rules = rule_performance[rule_performance['Pattern'] == pattern]
            recommendations.append({
                'Category': 'Pattern-Based Optimization',
                'Rules': f"{pattern} pattern rules",
                'Action': f"Review and adjust thresholds for {pattern} rules",
                'Rationale': f"Low average TP rate ({tp_rate:.1f}%) for {pattern} pattern",
                'Impact': 'Improve detection effectiveness for this pattern type'
            })
    
    # 5. Frequency-based recommendations
    daily_rules = rule_performance[(rule_performance['Frequency'] == 'daily') & 
                                    (rule_performance['TP_Rate'] < 30)].sort_values('Total', ascending=False)
    
    if not daily_rules.empty:
        recommendations.append({
            'Category': 'Frequency Adjustment',
            'Rules': ', '.join(daily_rules['alert_rules'].head(3)),
            'Action': 'Convert inefficient daily rules to weekly frequency',
            'Rationale': 'Daily rules generate high volume with low TP rate',
            'Impact': 'Reduce alert volume while potentially capturing more meaningful patterns'
        })
    
    # Convert to DataFrame
    recommendations_df = pd.DataFrame(recommendations)
    
    # Summary visualizations of recommendations
    if not recommendations_df.empty:
        plt.figure(figsize=(10, 6))
        recommendations_df['Category'].value_counts().plot(kind='barh', color='skyblue')
        plt.title('Recommendations by Category')
        plt.xlabel('Count')
        plt.tight_layout()
        plt.show()
    
    print(f"\nGenerated {len(recommendations_df)} recommendations for rule optimization")
    print(recommendations_df)
    
    return correlation_matrix, high_corr_pairs, recommendations_df

In [ ]:
# Execute the rule overlap and recommendations analysis
correlation_matrix, high_corr_pairs, recommendations = identify_rule_overlap_and_generate_recommendations(
    transaction_data, rule_descriptions, rule_performance, kyc_alerts
)

## 7. Save Results

Let's save our key findings to Excel for easier sharing and further analysis.

In [ ]:
# Save key results to Excel
with pd.ExcelWriter('terrapay_analysis_results.xlsx') as writer:
    # Rule performance
    rule_performance.to_excel(writer, sheet_name='Rule Performance', index=False)
    
    # True Positive cases
    true_positives.to_excel(writer, sheet_name='True Positives', index=False)
    
    # KYC breakage analysis
    pd.DataFrame({'sender_name': sender_name_groups['sender_name'], 
                 'kyc_count': sender_name_groups['kyc_id_count']}).to_excel(
        writer, sheet_name='Sender KYC Breakage', index=False)
    
    pd.DataFrame({'receiver_name': receiver_name_groups['receiver_name'], 
                 'kyc_count': receiver_name_groups['kyc_id_count']}).to_excel(
        writer, sheet_name='Receiver KYC Breakage', index=False)
    
    # Rule correlation matrix
    correlation_matrix.to_excel(writer, sheet_name='Rule Correlation Matrix')
    
    # Recommendations
    recommendations.to_excel(writer, sheet_name='Recommendations', index=False)

print("\nAnalysis complete. Results saved to 'terrapay_analysis_results.xlsx'.")
print("Key findings and recommendations have been generated based on the analysis.")

## 8. Conclusions and Next Steps

Based on our analysis, we can draw several conclusions:

1. **Rule Efficiency**: We've identified rules with low true positive rates that are generating excessive false positives.

2. **KYC Breakage Impact**: The issue of multiple KYC IDs for the same person is significantly affecting rule effectiveness, especially for receiver-focused rules.

3. **Rule Overlap**: Several rules are highly correlated and can be consolidated to reduce alert volume without losing effectiveness.

4. **Pattern and Frequency Effectiveness**: Different rule patterns and frequencies show varying levels of effectiveness, with some combinations performing better than others.

### Next Steps:

1. Implement the recommended rule optimizations, starting with the highest priority items

2. Develop a name/phone matching system to mitigate the KYC breakage issue

3. Monitor the impact of changes on overall true positive and false positive rates

4. Consider more advanced statistical models or machine learning approaches as a future enhancement

5. Establish a regular review process to continuously optimize the rules based on ongoing performance metrics